# How to Detect Random Walk and White Noise in Time Series Forecasting
## and how to deal with them if you find one   ## TODO
![](images/pexels.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@pripicart?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Tobi</a>
        on 
        <a href='https://www.pexels.com/photo/person-stands-on-brown-pathway-631986/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

## Setup

In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

In [4]:
tps_july = pd.read_csv(
    "https://raw.githubusercontent.com/BexTuychiev/medium_stories/master/2021/july/4_advanced_ts_visuals/data/tps_july_train.csv",
    parse_dates=["date_time"],
    index_col="date_time",
)

## Introduction

No matter how powerful, machine learning *cannot* predict everything. A well-known area where it can become pretty helpless is related to time series forecasting.

Despite the availability of a large suite of autoregressive models and many other algorithms for time series, you cannot predict the target distribution if it is **white noise** or follows a **random walk**. 

So, it is essential you detect such distributions before you make further efforts. 

In this article, you will learn what white noise and random walk are and explore proven statistical techniques to detect them.

## Before we start...

This is my third article on time series forecasting series (you check out the whole series from this [list](https://ibexorigin.medium.com/list/time-series-forecast-from-scratch-c72ccf7a9229), it is a new Medium feature).

While the first one was about every single Pandas function to manipulate TS data, the second one was about time series decomposition and autocorrelation. 

To get the most out of this article, you need to at least understand what autocorrelation is. Here, I will give a brief explanation but if you want to go deeper, check out my last article.